In [1]:
#-----------------------------------------------------------------------------------------------------------------
# Simulation experiment
#
# Network:
# A --> gene set C
#
# Algorithm
# Let gene A and B be TF's:
# if gene A expression > threshold_A:
#     genes in set C are upregulated by some percentage of its original expression
#
# Apply this algorithm for each sample in the compendium (essentially adding a signal to the existing gene expression data in the compendium)
# 
# Hyperparmeters should include: 
# 1. Gene A
# 2. Size of gene sets C
# 3. Thresholds
# 4. Percentage to 1.0 (effect size)
# 5. Log file with hyperparameter selections
#-----------------------------------------------------------------------------------------------------------------
%load_ext autoreload
%autoreload 2

import os
import pandas as pd
import numpy as np
import random

from functions import generate_input, vae, def_offset, interpolate, pca, plot

from numpy.random import seed
randomState = 5
seed(randomState)

Using TensorFlow backend.


In [2]:
# Hyperparameters

# Transcription factors
# If empty, genes will be randomly assigned
geneA = ''

# Size of the gene set that will be regulated by gene A
geneSetC_size = 1000 

# Threshold for activation of gene A 
thresholdA = 0.5

# Amount that genes in gene sets C 
effect_sizeA = 0.5

# Name of analysis
analysis_name = 'sim_nonlin_5'

In [3]:
# Load arguments
data_file = os.path.join(os.path.dirname(os.getcwd()), "data", "all-pseudomonas-gene-normalized.zip")

In [4]:
# Read in data
data = pd.read_table(data_file, header=0, sep='\t', index_col=0, compression='zip').T
data.head(5)

Gene_symbol,PA0001,PA0002,PA0003,PA0004,PA0005,PA0006,PA0007,PA0008,PA0009,PA0010,...,PA5561,PA5562,PA5563,PA5564,PA5565,PA5566,PA5567,PA5568,PA5569,PA5570
0.1_12hr_CSV86(Pae_G1a).CEL,0.472897,0.396658,0.253776,0.000000,0.175640,0.554385,0.411370,0.382222,0.310144,0.642522,...,0.358597,0.390048,0.457406,0.684082,0.338351,0.608325,0.643496,0.276075,0.112773,0.145170
0.1_2hr_CSV86(Pae_G1a).CEL,0.262346,0.086216,0.359853,0.439214,0.269749,0.768433,0.212505,0.062043,0.567695,0.467073,...,0.358504,0.414206,0.389879,0.477693,0.000000,0.479385,0.154471,0.140891,0.167505,0.157060
0.1_6hr_CSV86(Pae_G1a).CEL,0.473658,0.244862,0.330750,0.097697,0.387226,0.328319,0.228820,0.330039,0.318081,0.512864,...,0.180744,0.380741,0.173501,0.251571,0.182793,0.528301,0.504985,0.499782,0.061106,0.365612
0.1_7hr_CSV86(Pae_G1a).CEL,0.439273,0.343402,0.192698,0.274677,0.628979,0.553796,0.431391,0.363480,0.385721,0.094584,...,0.346837,0.153927,0.067349,0.319723,0.282442,0.490655,0.531415,0.153880,0.132333,0.260087
0.1_9hr_CSV86(Pae_G1a).CEL,0.220827,0.145525,0.437803,0.293201,0.635120,0.462893,0.488733,0.309584,0.318646,0.591914,...,0.237726,0.301945,0.070222,0.513605,0.114277,0.360259,0.386868,0.223995,0.105343,0.102088


In [5]:
# Randomly select gene A if not specified
# Note: 'replace=False' indicates sampling WITHOUT replacement
if not geneA:
    gene_ids = list(data.columns)
    [geneA] = np.random.choice(gene_ids, size=1, replace=False)

print(geneA)

PA2845


In [6]:
# checkpoint
assert(len(gene_ids)==5549)

In [7]:
# Randomly select genes for gene set C
# remove() doesn't return a value it will remove the element from the list object
gene_ids.remove(geneA)
print(len(gene_ids))

# Random sample of genes for set C
geneSetC = random.sample(gene_ids, geneSetC_size)

print(len(geneSetC))

5548
1000


In [8]:
# checkpoint
assert(geneA not in geneSetC)

In [9]:
# checkpoint
# print(data[geneA])

0.1_12hr_CSV86(Pae_G1a).CEL           0.561571
0.1_2hr_CSV86(Pae_G1a).CEL            0.619182
0.1_6hr_CSV86(Pae_G1a).CEL            0.367484
0.1_7hr_CSV86(Pae_G1a).CEL            0.120485
0.1_9hr_CSV86(Pae_G1a).CEL            0.598280
05_PA14000-4-2_5-10-07_S2.CEL         0.073053
1_12hr_CSV86(Pae_G1a).CEL             0.469162
1_2hr_CSV86(Pae_G1a).CEL              0.445728
1_6hr_CSV86(Pae_G1a).CEL              0.373592
1_7hr_CSV86(Pae_G1a).CEL              0.338715
1_9hr_CSV86(Pae_G1a).CEL              0.168722
120330-10978D_22_(Pae_G1a).CEL        0.126023
120330-10978D_23_(Pae_G1a).CEL        0.150161
120330-10978D_24_(Pae_G1a).CEL        0.215326
120330-10978D_M1_(Pae_G1a).CEL        0.177812
120330-10978D_M3_(Pae_G1a).CEL        0.130992
120330-10978D_M4_(Pae_G1a).CEL        0.147811
142_2014wt_anaero.CEL                 0.075467
143b_2014wt_anaero.CEL                0.076775
144_2014wt_anaero.CEL                 0.142544
145_2014dR_anaero.CEL                 0.111962
146_2014dR_an

In [10]:
# checkpoint: before transformation
# data.loc[data[geneA]<=thresholdA,geneSetC[0]]

0.1_6hr_CSV86(Pae_G1a).CEL            0.711318
0.1_7hr_CSV86(Pae_G1a).CEL            0.709582
05_PA14000-4-2_5-10-07_S2.CEL         0.356762
1_12hr_CSV86(Pae_G1a).CEL             0.683754
1_2hr_CSV86(Pae_G1a).CEL              0.837280
1_6hr_CSV86(Pae_G1a).CEL              0.268017
1_7hr_CSV86(Pae_G1a).CEL              0.910602
1_9hr_CSV86(Pae_G1a).CEL              0.660359
120330-10978D_22_(Pae_G1a).CEL        0.334563
120330-10978D_23_(Pae_G1a).CEL        0.385742
120330-10978D_24_(Pae_G1a).CEL        0.382716
120330-10978D_M1_(Pae_G1a).CEL        0.412453
120330-10978D_M3_(Pae_G1a).CEL        0.335547
120330-10978D_M4_(Pae_G1a).CEL        0.297483
142_2014wt_anaero.CEL                 0.325167
143b_2014wt_anaero.CEL                0.337707
144_2014wt_anaero.CEL                 0.290684
145_2014dR_anaero.CEL                 0.371489
146_2014dR_anaero.CEL                 0.311563
147_2014dR_anaero.CEL                 0.419222
20mM_KT2440.CEL                       0.632622
2mM_CF600.CEL

In [11]:
# Loop through all samples
num_samples = data.shape[1]

for sample_id in data.index:
    row = data.loc[sample_id]
        
    if data.loc[sample_id,geneA] > thresholdA:
        # Scale genes by some fixed percentage
        for gene in geneSetC:
            data.loc[sample_id,gene] = data.loc[sample_id,gene]*(1+effect_sizeA)            
            
# if any exceed 1 then set to 1 since gene expression is normalized
data[data>=1.0] = 1.0

0.1_12hr_CSV86(Pae_G1a).CEL
0.1_2hr_CSV86(Pae_G1a).CEL
0.1_9hr_CSV86(Pae_G1a).CEL
20mM_CF600.CEL
GSM1112047_PGAS_25_2A.CEL
GSM1553191_Normal_C1.CEL
GSM1608069_Biofilm+12hrcipro_rep3.CEL
GSM1608071_Biofilm+12hrtobra_rep2.CEL
GSM1608072_Biofilm+12hrtobra_rep3.CEL
GSM197238.CEL
GSM234929.CEL
GSM537320.CEL
GSM562113_Koh_PAK_H2O_2.CEL
GSM596626.CEL
GSM596819.CEL
GSM625973.CEL
GSM625974.CEL
GSM625975.CEL
GSM625976.CEL
GSM625977.CEL
GSM743008_PAO1-SC24-b.CEL
GSM807323.CEL
GSM807324.CEL
GSM821087_HZI0142_Pae_G1a.CEL
GSM821088_HZI0143_Pae_G1a.CEL
GSM821089_HZI0144_Pae_G1a.CEL
GSM823344.CEL
GSM871967.CEL
GSM871976.CEL
GSM871978.CEL
GSM900177_F1_Pae_G1a_.CEL
GSM900179_F2_Pae_G1a_.CEL
JS-1B4.9.07.CEL
JS-A164.9.07.CEL
JS-G84.18.07.CEL
PA01_Control_1.CEL
Pae_Fe02_15.CEL
PAO1_Control_1.CEL


In [12]:
# checkpoint: after transformation
# data.loc[data[geneA]<=thresholdA,geneSetC[0]]

0.1_6hr_CSV86(Pae_G1a).CEL            0.711318
0.1_7hr_CSV86(Pae_G1a).CEL            0.709582
05_PA14000-4-2_5-10-07_S2.CEL         0.356762
1_12hr_CSV86(Pae_G1a).CEL             0.683754
1_2hr_CSV86(Pae_G1a).CEL              0.837280
1_6hr_CSV86(Pae_G1a).CEL              0.268017
1_7hr_CSV86(Pae_G1a).CEL              0.910602
1_9hr_CSV86(Pae_G1a).CEL              0.660359
120330-10978D_22_(Pae_G1a).CEL        0.334563
120330-10978D_23_(Pae_G1a).CEL        0.385742
120330-10978D_24_(Pae_G1a).CEL        0.382716
120330-10978D_M1_(Pae_G1a).CEL        0.412453
120330-10978D_M3_(Pae_G1a).CEL        0.335547
120330-10978D_M4_(Pae_G1a).CEL        0.297483
142_2014wt_anaero.CEL                 0.325167
143b_2014wt_anaero.CEL                0.337707
144_2014wt_anaero.CEL                 0.290684
145_2014dR_anaero.CEL                 0.371489
146_2014dR_anaero.CEL                 0.311563
147_2014dR_anaero.CEL                 0.419222
20mM_KT2440.CEL                       0.632622
2mM_CF600.CEL

In [13]:
# Dataframe with only gene A
geneA_only = pd.DataFrame(data[geneA], index=data.index, columns=[geneA])

# Drop gene A
data_holdout = data.drop(columns=[geneA])

In [14]:
# checkpoint
data_holdout.shape

(1191, 5548)

In [15]:
# Create list of base directories
base_dirs = [os.path.join(os.path.dirname(os.getcwd()), 'data'),
             os.path.join(os.path.dirname(os.getcwd()), 'encoded'),
             os.path.join(os.path.dirname(os.getcwd()), 'models'),
             os.path.join(os.path.dirname(os.getcwd()), 'output'),
             os.path.join(os.path.dirname(os.getcwd()), 'stats'),
             os.path.join(os.path.dirname(os.getcwd()), 'viz')    
]

# Check if directory exist otherwise create
for each_dir in base_dirs:
    analysis_dir = os.path.join(each_dir, analysis_name)
    if os.path.exists(analysis_dir):
        print('directory already exists: {}'.format(analysis_dir))
    else:
        os.mkdir(analysis_dir)
        print('creating new directory: {}'.format(analysis_dir))

directory already exists: /home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/data/sim_nonlin_5
directory already exists: /home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/encoded/sim_nonlin_5
directory already exists: /home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/models/sim_nonlin_5
directory already exists: /home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/output/sim_nonlin_5
directory already exists: /home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/stats/sim_nonlin_5
directory already exists: /home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/viz/sim_nonlin_5


In [16]:
# Output the new gene expression values for each sample
train_input_file = os.path.join(base_dirs[0], analysis_name, "train_model_input.txt.xz")
data_holdout.to_csv(train_input_file, sep='\t', compression='xz', float_format="%.5g")

# Output log file with params
log_file = os.path.join(os.path.dirname(os.getcwd()), 'metadata', analysis_name)

args_dict = {
    "gene A": geneA,
    "size of gene set C": geneSetC_size,
    "genes in set C": geneSetC,
    "threshold of A activation": thresholdA,
    "effect size of genes in set C": effect_sizeA,
}

with open(log_file, 'w') as f:
    for key, value in args_dict.items():
        f.write('%s: %s\n' % (key, value))
        
# Output geneA only file
geneA_file = os.path.join(base_dirs[0], analysis_name, geneA + ".txt")
geneA_only.to_csv(geneA_file, sep='\t', float_format="%.5g")

In [17]:
%time
# Train models on input dataset

# params
learning_rate = 0.001
batch_size = 100
epochs = 200
kappa = 0.01
intermediate_dim = 100
latent_dim = 10
epsilon_std = 1.0
num_PCs = latent_dim

base_dir = os.path.dirname(os.getcwd())
vae.tybalt_2layer_model(learning_rate, batch_size, epochs, kappa, intermediate_dim, latent_dim, epsilon_std, base_dir, analysis_name)
pca.pca_model(base_dir, analysis_name, num_PCs)


# Define offset vectors in gene space
data_dir = os.path.join(base_dirs[0], analysis_name)
target_gene = geneA
percent_low = 5
percent_high = 95

def_offset.gene_space_offset(data_dir, target_gene, percent_low, percent_high)


# Define offset vectors for different latent spaces
model_dir = os.path.join(base_dirs[2], analysis_name)
encoded_dir = os.path.join(base_dirs[1], analysis_name)

def_offset.vae_latent_space_offset(data_dir, model_dir, encoded_dir, target_gene, percent_low, percent_high)
def_offset.pca_latent_space_offset(data_dir, model_dir, encoded_dir, target_gene, percent_low, percent_high)


# Predict gene expression using offset in gene space and latent space
out_dir = os.path.join(base_dirs[3], analysis_name)

interpolate.interpolate_in_gene_space(data_dir, target_gene, out_dir, percent_low, percent_high)
interpolate.interpolate_in_vae_latent_space(data_dir, model_dir, encoded_dir, target_gene, out_dir, percent_low, percent_high)
interpolate.interpolate_in_pca_latent_space(data_dir, model_dir, encoded_dir, target_gene, out_dir, percent_low, percent_high)


# Plot prediction per sample along gradient of PA1673 expression
viz_dir = os.path.join(base_dirs[5], analysis_name)
plot.plot_corr_gradient(out_dir, viz_dir)


CPU times: user 1 µs, sys: 0 ns, total: 1 µs
Wall time: 4.05 µs
Number of genes in low expression group is (60, 5548)
Number of gene in high expression group is (61, 5548)


/home/alexandra/anaconda3/envs/Pa/lib/python3.5/site-packages/keras/models.py:255: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


Number of genes in low expression group is (60, 5548)
Number of gene in high expression group is (61, 5548)
Number of genes in low expression group is (60, 5548)
Number of gene in high expression group is (61, 5548)
                                       gene_space  latent_space_vae  \
GSM546254.CEL                            0.902391          0.919959   
Overhage13002_(Pae_G1a).CEL              0.795629          0.767433   
GSM546274.CEL                            0.891857          0.895358   
GSM432863.CEL                            0.894534          0.882666   
GSM774122_CF333-1997b.CEL                0.893202          0.880890   
GSM774124_CF333-2003a.CEL                0.884424          0.868483   
GSM1420996_anr_NO1.CEL                   0.915233          0.896310   
GSM774121_CF333-1997a.CEL                0.894011          0.883930   
GSM1420995_anr_control2.CEL              0.932701          0.909875   
GSM1889279_L-Hyp_02.CEL                  0.852970          0.853778   
GSM

/home/alexandra/Documents/Repos/Pseudomonas_latent_spaces/scripts/functions/plot.py:52: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  X_sorted = X.loc[sorted_id.index].dropna()
